In [13]:
# CIFAR-100 다운로드 및 로딩
from common.util import shuffle_dataset
import pickle, os
import numpy as np

def load_batch(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
    return data[b'data'], np.array(data[b'fine_labels']), np.array(data[b'coarse_labels'])

def normalize(x):
    mean = np.array([0.507, 0.487, 0.441]).reshape(1, 3, 1, 1)
    std = np.array([0.267, 0.256, 0.276]).reshape(1, 3, 1, 1)
    return (x - mean) / std

def load_cifar100():
    x_train, y_train_fine, y_train_coarse = load_batch("./cifar-100-python/train")
    x_test, y_test_fine, y_test_coarse = load_batch("./cifar-100-python/test")
    x_test = x_test.reshape(-1, 3, 32, 32).astype(np.float32) / 255.0
    x_test = normalize(x_test)
    return x_test, y_test_fine, y_test_coarse

x_test, y_test_fine, y_test_coarse = load_cifar100()

In [14]:
def create_normalized_fine_to_coarse_matrix(fine_labels, coarse_labels, num_fine=100, num_coarse=20):
    count = np.zeros((num_fine, num_coarse), dtype=np.float32)
    for fine, coarse in zip(fine_labels, coarse_labels):
        count[fine, coarse] += 1
    row_sums = np.sum(count, axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1  # 방어적 처리
    return count / row_sums  # normalize to soft weights

In [15]:
# train에서 label 관계 추출
_, y_train_fine, y_train_coarse = load_batch("./cifar-100-python/train")
mapping_matrix = create_normalized_fine_to_coarse_matrix(y_train_fine, y_train_coarse)

In [16]:
import numpy as np
from common.functions import softmax
from common.layers import Convolution, Affine, Relu, BatchNormalization
from common.util import im2col, col2im

# --- Affine (양자화 제거) ---
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + self.b
        return out

# --- Convolution (양자화 제거) ---
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        self.x = None
        self.col = None
        self.col_W = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, _, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        return out

# --- Residual Block ---
class ResidualBlock:
    def __init__(self, in_channels, out_channels, stride=1):
        self.stride = stride
        self.equal_in_out = (in_channels == out_channels and stride == 1)

        self.conv1 = Convolution(np.random.randn(out_channels, in_channels, 3, 3), np.zeros(out_channels), stride=stride, pad=1)
        self.bn1 = BatchNormalization(np.ones(out_channels), np.zeros(out_channels))
        self.relu1 = Relu()

        self.conv2 = Convolution(np.random.randn(out_channels, out_channels, 3, 3), np.zeros(out_channels), stride=1, pad=1)
        self.bn2 = BatchNormalization(np.ones(out_channels), np.zeros(out_channels))
        self.relu2 = Relu()

        if not self.equal_in_out:
            self.shortcut = Convolution(np.random.randn(out_channels, in_channels, 1, 1), np.zeros(out_channels), stride=stride, pad=0)
            self.bn_shortcut = BatchNormalization(np.ones(out_channels), np.zeros(out_channels))

    def forward(self, x, train_flg=False):
        out = self.relu1.forward(self.bn1.forward(self.conv1.forward(x), train_flg))
        out = self.bn2.forward(self.conv2.forward(out), train_flg)

        shortcut = x if self.equal_in_out else self.bn_shortcut.forward(self.shortcut.forward(x), train_flg)
        out += shortcut
        return self.relu2.forward(out)

# --- ResNet-20 ---
class ResNet20:
    def __init__(self, num_classes=100):
        self.conv1 = Convolution(np.random.randn(16, 3, 3, 3), np.zeros(16), stride=1, pad=1)
        self.bn1 = BatchNormalization(np.ones(16), np.zeros(16))
        self.relu1 = Relu()

        self.layer1 = [ResidualBlock(16, 16, stride=1) for _ in range(3)]
        self.layer2 = [ResidualBlock(16 if i == 0 else 32, 32, stride=2 if i == 0 else 1) for i in range(3)]
        self.layer3 = [ResidualBlock(32 if i == 0 else 64, 64, stride=2 if i == 0 else 1) for i in range(3)]

        self.fc = Affine(np.random.randn(64, num_classes), np.zeros(num_classes))

    def forward(self, x):
        out = self.relu1.forward(self.bn1.forward(self.conv1.forward(x), train_flg=False))
        for block in self.layer1: out = block.forward(out)
        for block in self.layer2: out = block.forward(out)
        for block in self.layer3: out = block.forward(out)

        out = out.mean(axis=(2, 3))  # Global Average Pooling
        return self.fc.forward(out)

    def predict(self, x, batch_size=100):
        return np.concatenate([self.forward(x[i:i+batch_size]) for i in range(0, x.shape[0], batch_size)], axis=0)

In [18]:
from common.functions import softmax

# 저장된 모델 불러오기
with open("ANN_Generalized/ResNet20_cfg1_smooth0.05_epoch_10.pkl", 'rb') as f:
    model_data = pickle.load(f)

model = ResNet20()

# 저장된 weight 불러오기
param_dict = model_data['model']
idx = 0
for layer in model.layer1 + model.layer2 + model.layer3:
    for attr in ['conv1', 'conv2', 'shortcut']:
        if hasattr(layer, attr) and f"{idx}_W" in param_dict:
            conv = getattr(layer, attr)
            conv.W = param_dict[f"{idx}_W"]
            conv.b = param_dict[f"{idx}_b"]
            idx += 1
model.fc.W = param_dict['fc_W']
model.fc.b = param_dict['fc_b']

In [19]:
def compute_coarse_accuracy(model, x, y_coarse, mapping_matrix, batch_size=100):
    correct = 0
    total = x.shape[0]

    for i in range(0, total, batch_size):
        x_batch = x[i:i+batch_size]
        y_batch = y_coarse[i:i+batch_size]

        fine_logits = model.predict(x_batch)  # (N, 100)
        fine_probs = softmax(fine_logits)     # (N, 100)

        coarse_probs = np.dot(fine_probs, mapping_matrix)  # (N, 20)
        pred_coarse = np.argmax(coarse_probs, axis=1)

        correct += np.sum(pred_coarse == y_batch)

    return correct / total

In [ ]:
coarse_acc = compute_coarse_accuracy(model, x_test, y_test_coarse, mapping_matrix)
print(f"Coarse label accuracy: {coarse_acc:.4f}")

✅ Coarse label accuracy: 0.0505


In [21]:
fine_logits = model.predict(x_test)
fine_logits -= np.max(fine_logits, axis=1, keepdims=True)  # overflow 방지
coarse_logits = fine_logits @ fine_to_coarse_matrix
coarse_probs = softmax(coarse_logits)
coarse_preds = np.argmax(coarse_probs, axis=1)
accuracy = np.mean(coarse_preds == y_test_coarse)

NameError: name 'fine_to_coarse_matrix' is not defined

In [26]:
import numpy as np
import pickle
from common.functions import softmax

# 1. CIFAR-100 coarse 라벨 메타 정보 불러오기
def load_meta(data_dir="./cifar-100-python"):
    with open(f"{data_dir}/meta", 'rb') as f:
        meta = pickle.load(f, encoding='bytes')
        return meta[b'fine_label_names'], meta[b'coarse_label_names']

# 2. CIFAR-100 test 데이터 로딩
def load_cifar100_test(data_dir="./cifar-100-python"):
    with open(f"{data_dir}/test", 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
    x = batch[b'data'].reshape(-1, 3, 32, 32).astype(np.float32) / 255.0
    mean = np.array([0.507, 0.487, 0.441]).reshape(1, 3, 1, 1)
    std = np.array([0.267, 0.256, 0.276]).reshape(1, 3, 1, 1)
    x = (x - mean) / std
    y_fine = np.array(batch[b'fine_labels'])
    y_coarse = np.array(batch[b'coarse_labels'])
    return x, y_fine, y_coarse

# 3. fine → coarse 매핑 행렬 생성
# 100개 fine label → 20개 coarse label 매핑 행렬 생성
def create_fine_to_coarse_matrix_from_meta(meta_path="./cifar-100-python/train", num_fine=100, num_coarse=20):
    with open(meta_path, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
        fine_labels = batch[b'fine_labels']
        coarse_labels = batch[b'coarse_labels']

    mapping = np.zeros((num_fine, num_coarse), dtype=np.float32)
    for fine, coarse in zip(fine_labels, coarse_labels):
        mapping[fine, coarse] = 1.0

    return mapping

# 4. 저장된 모델 파라미터 로드 & 적용
def load_weights_into_model(model, pkl_path):
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    weights = data['model']
    
    # fc layer
    model.fc.W = weights['fc_W']
    model.fc.b = weights['fc_b']

    # conv layers
    conv_layers = (
        model.layer1 + model.layer2 + model.layer3
    )
    i = 0
    for layer in conv_layers:
        for conv_name in ['conv1', 'conv2']:
            conv = getattr(layer, conv_name)
            conv.W = weights[f'{i}_W']
            conv.b = weights[f'{i}_b']
            i += 1
        if hasattr(layer, 'shortcut'):
            layer.shortcut.W = weights[f'{i}_W']
            layer.shortcut.b = weights[f'{i}_b']
            i += 1
    print("✅ Model weights loaded.")

# 5. 추론 및 coarse accuracy 계산
def evaluate_coarse_accuracy(model, x_test, y_coarse, fine_to_coarse_mat):
    batch_size = 100
    correct = 0
    total = x_test.shape[0]

    for i in range(0, total, batch_size):
        x_batch = x_test[i:i+batch_size]
        y_batch = y_coarse[i:i+batch_size]

        # logits: (B, 100)
        fine_logits = model.predict(x_batch)
        fine_logits -= np.max(fine_logits, axis=1, keepdims=True)  # stability
        coarse_logits = fine_logits @ fine_to_coarse_mat  # (B, 20)
        coarse_probs = softmax(coarse_logits)
        coarse_preds = np.argmax(coarse_probs, axis=1)

        correct += np.sum(coarse_preds == y_batch)

    acc = correct / total
    print(f"✅ Coarse Accuracy: {acc:.4f}")
    return acc

In [29]:
x_test, y_fine, y_coarse = load_cifar100_test()
fine_names, coarse_names = load_meta()
fine_to_coarse = create_fine_to_coarse_matrix_from_meta()

model = ResNet20(num_classes=100)
load_weights_into_model(model, "ANN_Generalized/ResNet20_cfg1_smooth0.05_epoch_10.pkl")

evaluate_coarse_accuracy(model, x_test, y_coarse, fine_to_coarse)

✅ Model weights loaded.
✅ Coarse Accuracy: 0.0500


np.float64(0.05)